# Flow NetCDF
Since we're now using NetCDF files instead of png, we will need to modify a lot of the functions we used to visualize our result.

## Importing necessary libraries and notebooks

In [2]:
import xarray as xr
import io
import os
import cv2
import imageio
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import netCDF4 as nc
import scipy.stats
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import time
from datetime import datetime, timedelta
from matplotlib import ticker
from matplotlib.lines import Line2D
from matplotlib.patches import FancyArrowPatch
from matplotlib.collections import LineCollection
from IPython.display import Image, display, clear_output
from PIL import Image as PILImage
import concurrent.futures

# Import the other notebooks without running their cells
from ii_Data_Manipulation import visualize_4
from vii_Flow_Analysis import haversine

## NetCDF Functions

### *visualize*
A generalization of the visualize function in the very first notebook with the added option of plotting flow vectors and choosing the step (density) and scale of the flow vectors.

In [ ]:
def visualize(file_path, variable_key="fai_anomaly", lat_range=None, lon_range=None, color="viridis", colorbar_label="", title="", flow_vectors=None, quiver_step=None, quiver_scale=None):
    """
    Visualizes the NetCDF data and optionally overlays flow vectors.

    Parameters:
    - file_path: Path to the NetCDF file.
    - variable_key: Key for the variable of interest in the NetCDF dataset.
    - lat_range: Tuple of (min, max) latitude to subset the data.
    - lon_range: Tuple of (min, max) longitude to subset the data.
    - color: Color map for the plot.
    - colorbar_label: Label for the color bar.
    - title: Title of the plot.
    - flow_vectors: Optional tuple of flow vector components (flow_u, flow_v).
    - quiver_step: Sampling step for displaying quiver arrows, controls density.
    - quiver_scale: Scaling factor for quiver arrows, controls size.
    """
    # Load the netCDF data
    data = xr.open_dataset(file_path)
    
    # If ranges are specified, apply them to select the desired subset
    if lat_range and 'latitude' in data.coords:
        data = data.sel(latitude=slice(*lat_range))
    if lon_range and 'longitude' in data.coords:
        data = data.sel(longitude=slice(*lon_range))

    # Set up a plot with geographic projections
    fig, ax = plt.subplots(figsize=(12, 10), subplot_kw={'projection': ccrs.PlateCarree()})
    
    # Extract relevant data 
    index_data = data[variable_key]

    # Plot the data
    im = index_data.plot(ax=ax, x='longitude', y='latitude', transform=ccrs.PlateCarree(),
                         cmap=color, add_colorbar=True, extend='both', cbar_kwargs={'shrink': 0.35})

    # Add color bar details
    im.colorbar.set_label(colorbar_label)

    # Customize the map with coastlines and features
    ax.coastlines(resolution='10m', color='black')
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    gl = ax.gridlines(draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False

    # Plot flow vectors if provided
    if flow_vectors:
        # Automatically determine quiver_step and quiver_scale if not provided
        if quiver_step is None:
            quiver_step = max(1, int(len(data.latitude) / 20))  # Sample about 20 arrows along the latitude
        if quiver_scale is None:
            quiver_scale = max(1, int(len(data.latitude) / 2))  # Scale according to number of latitude points

        # Create a meshgrid for the flow vectors that matches the data subset
        Y, X = np.meshgrid(data.latitude, data.longitude, indexing='ij')
        # Apply the step for vector density and scale for vector size
        ax.quiver(X[::quiver_step, ::quiver_step], Y[::quiver_step, ::quiver_step], flow_vectors[0][::quiver_step, ::quiver_step], flow_vectors[1][::quiver_step, ::quiver_step], color='red', scale=quiver_scale)

    # Show the plot with title
    plt.title(title)
    plt.show()

### *calculate_deepflow*
A function to calculate and return deepflow using as input two NetCDF files.

In [ ]:
def calculate_deepflow(nc_file1, nc_file2, variable_key="fai_anomaly", time_interval=86400):
    # Load data
    data1 = xr.open_dataset(nc_file1)
    data2 = xr.open_dataset(nc_file2)
    img1 = data1[variable_key].values
    img2 = data2[variable_key].values

    # Ensure data is 2D
    if img1.ndim == 3:
        img1 = img1[0]
    if img2.ndim == 3:
        img2 = img2[0]

    # Normalize and convert to 8-bit grayscale
    img1 = cv2.normalize(img1, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    img2 = cv2.normalize(img2, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Compute DeepFlow
    deep_flow = cv2.optflow.createOptFlow_DeepFlow()
    flow = deep_flow.calc(img1, img2, None)

    # Get latitude and longitude values
    latitudes = data1.latitude.values
    longitudes = data1.longitude.values

    # Calculate consistent distances between consecutive latitudes and longitudes
    d_lat_m = haversine(longitudes[0], latitudes[0], longitudes[0], latitudes[1]) * 1000
    d_lon_m = haversine(longitudes[0], latitudes[0], longitudes[1], latitudes[0]) * 1000

    # Convert pixel flow to real-world distance flow in meters per second
    flow_u_mps = flow[..., 0] * (d_lon_m / time_interval)
    flow_v_mps = flow[..., 1] * (d_lat_m / time_interval)

    return flow_u_mps, flow_v_mps  # Return flow in meters per second

### *calculate_farneback*

In [ ]:
def calculate_farneback(nc_file1, nc_file2, variable_key="fai_anomaly", time_interval=86400):
    # Load data
    data1 = xr.open_dataset(nc_file1)
    data2 = xr.open_dataset(nc_file2)
    img1 = data1[variable_key].values
    img2 = data2[variable_key].values

    # Get the latitude and longitude values from the dataset
    latitudes = data1.latitude.values
    longitudes = data1.longitude.values

    # Calculate consistent distances between consecutive latitudes and longitudes
    d_lat_m = haversine(longitudes[0], latitudes[0], longitudes[0], latitudes[1]) * 1000
    d_lon_m = haversine(longitudes[0], latitudes[0], longitudes[1], latitudes[0]) * 1000

    # Ensure data is 2D
    if img1.ndim == 3:
        img1 = img1[0]
    if img2.ndim == 3:
        img2 = img2[0]

    # Normalize and convert to 8-bit grayscale
    img1 = cv2.normalize(img1, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    img2 = cv2.normalize(img2, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Compute Farneback Optical Flow
    flow = cv2.calcOpticalFlowFarneback(img1, img2, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Convert pixel flow to real-world distance flow in meters per second
    flow_u_mps = flow[..., 0] * (d_lon_m / time_interval)
    flow_v_mps = flow[..., 1] * (d_lat_m / time_interval)

    return flow_u_mps, flow_v_mps  # Return flow in meters per second

### *calculate_lk*

In [ ]:
def calculate_lk(nc_file1, nc_file2, variable_key="fai_anomaly"):
    # Load data
    data1 = xr.open_dataset(nc_file1)
    data2 = xr.open_dataset(nc_file2)
    img1 = data1[variable_key].values
    img2 = data2[variable_key].values

    # Compute distances in meters between consecutive latitude and longitude points
    latitudes = data1.latitude.values
    longitudes = data1.longitude.values
    d_lat_km = haversine(longitudes[0], latitudes[0], longitudes[0], latitudes[1])
    d_lon_km = haversine(longitudes[0], latitudes[0], longitudes[1], latitudes[0])

    # Ensure data is 2D
    if img1.ndim > 2:
        img1 = img1.squeeze()  # Reduce dimensions if necessary
    if img2.ndim > 2:
        img2 = img2.squeeze()

    # Normalize and convert to 8-bit grayscale
    img1 = cv2.normalize(img1.astype(np.float32), None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    img2 = cv2.normalize(img2.astype(np.float32), None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Detect good features to track
    p0 = cv2.goodFeaturesToTrack(img1, maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
    if p0 is not None:
        # Calculate optical flow using Lucas-Kanade method
        p1, st, err = cv2.calcOpticalFlowPyrLK(img1, img2, p0, None)
        good_new = p1[st == 1] if p1 is not None else np.empty((0, 2))
        good_old = p0[st == 1]
    else:
        good_new = np.empty((0, 2))
        good_old = np.empty((0, 2))

    if good_new.size > 0:
        # Calculate flow in kilometers
        flow_u_km = (good_new[:, 0] - good_old[:, 0]) * (d_lon_km / len(longitudes))
        flow_v_km = (good_new[:, 1] - good_old[:, 1]) * (d_lat_km / len(latitudes))
    else:
        flow_u_km = np.array([])
        flow_v_km = np.array([])

    return flow_u_km, flow_v_km  # Return flow in kilometers per pixel

### *calculate_block_matching*

In [8]:
def calculate_block_matching(nc_file1, nc_file2, variable_key="fai_anomaly", time_interval=86400, block_size=(16, 16), search_area=(32, 32)):
    """
    Calculate flow using block matching between two images from NetCDF files.

    Parameters:
    - nc_file1: Path to the first NetCDF file.
    - nc_file2: Path to the second NetCDF file.
    - variable_key: Key to extract the relevant variable from NetCDF files.
    - time_interval: Time interval in seconds between the two images.
    - block_size: Size of each block (height, width).
    - search_area: Size of the area to search for a match around the block's original position.
    """
    # Load data
    data1 = xr.open_dataset(nc_file1)
    data2 = xr.open_dataset(nc_file2)
    img1 = data1[variable_key].values
    img2 = data2[variable_key].values

    # Ensure data is 2D
    if img1.ndim == 3:
        img1 = img1[0]
    if img2.ndim == 3:
        img2 = img2[0]

    # Normalize and convert to 8-bit grayscale
    img1 = cv2.normalize(img1, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    img2 = cv2.normalize(img2, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Flow vectors
    flow_u = np.zeros_like(img1, dtype=np.float32)
    flow_v = np.zeros_like(img1, dtype=np.float32)

    # Block matching algorithm
    for y in range(0, img1.shape[0], block_size[1]):
        for x in range(0, img1.shape[1], block_size[0]):
            block = img1[y:y+block_size[1], x:x+block_size[0]]
            # Define search area in img2
            search_start_x = max(x - search_area[0]//2, 0)
            search_end_x = min(x + search_area[0]//2 + block_size[0], img2.shape[1])
            search_start_y = max(y - search_area[1]//2, 0)
            search_end_y = min(y + search_area[1]//2 + block_size[1], img2.shape[0])

            search_region = img2[search_start_y:search_end_y, search_start_x:search_end_x]

            # Apply template matching
            result = cv2.matchTemplate(search_region, block, cv2.TM_SQDIFF_NORMED)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
            best_match_top_left = min_loc
            best_match_center = (best_match_top_left[0] + block_size[0] // 2, best_match_top_left[1] + block_size[1] // 2)

            # Calculate flow vectors
            flow_u[y:y+block_size[1], x:x+block_size[0]] = best_match_center[0] + search_start_x - x - block_size[0] // 2
            flow_v[y:y+block_size[1], x:x+block_size[0]] = best_match_center[1] + search_start_y - y - block_size[1] // 2

    # Normalize flow vectors to meters per second
    latitudes = data1.latitude.values
    longitudes = data1.longitude.values
    d_lat_m = haversine(longitudes[0], latitudes[0], longitudes[0], latitudes[1]) * 1000
    d_lon_m = haversine(longitudes[0], latitudes[0], longitudes[1], latitudes[0]) * 1000

    flow_u_mps = flow_u * (d_lon_m / time_interval)
    flow_v_mps = flow_v * (d_lat_m / time_interval)

    return flow_u_mps, flow_v_mps

### Default Color (Viridis)

In [ ]:
# Without Flow
if __name__ == "__main__":
    file_path = '/media/yahia/ballena/ABI/NetCDF/Partition/n = 24/Averages/[14.333333333333334,15.5],[-63.33333333333333,-67.0]/algae_distribution_20220723.nc'
    visualize(file_path)

In [ ]:
# Flow on Antilles
if __name__ == "__main__":
    prev_nc = '/home/yahia/Documents/Jupyter/Sargassum/Images/Test/ABI_Averages/algae_distribution_20220723.nc'
    next_nc = '/home/yahia/Documents/Jupyter/Sargassum/Images/Test/ABI_Averages/algae_distribution_20220724.nc'
    
    flow_vectors = calculate_deepflow(prev_nc, next_nc)
    visualize(prev_nc, variable_key="fai_anomaly", colorbar_label="FAI", title="Optical Flow", flow_vectors=flow_vectors)

In [ ]:
# Flow on Atlantic
if __name__ == "__main__":
    prev_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Averages/algae_distribution_20220723.nc"
    next_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Averages/algae_distribution_20220724.nc"
    
    flow_vectors = calculate_deepflow(prev_nc, next_nc)
    visualize(prev_nc, variable_key="fai_anomaly", colorbar_label="FAI", title="Optical Flow", flow_vectors=flow_vectors)

### Binarized

In [ ]:
# Without Flow
if __name__ == "__main__":
    file_path = '/media/yahia/ballena/ABI/NetCDF/Partition/n = 24/Averages_Binarized/[14.333333333333334,15.5],[-63.33333333333333,-67.0]/Processed_algae_distribution_20220723.nc'
    visualize(file_path)

In [ ]:
# Flow on Antilles
if __name__ == "__main__":
    prev_nc = '/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Processed_ABI_Averages/Processed_algae_distribution_20220723.nc'
    next_nc = '/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Processed_ABI_Averages/Processed_algae_distribution_20220724.nc'
    
    flow_vectors = calculate_deepflow(prev_nc, next_nc)
    visualize(prev_nc, variable_key="fai_anomaly", colorbar_label="FAI", title="Optical Flow", flow_vectors=flow_vectors)

In [ ]:
# Flow on Atlantic
if __name__ == "__main__":
    prev_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Averages_Binarized/Processed_algae_distribution_20220723.nc"
    next_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Averages_Binarized/Processed_algae_distribution_20220724.nc"
    
    flow_vectors = calculate_deepflow(prev_nc, next_nc)
    visualize(prev_nc, variable_key="fai_anomaly", colorbar_label="FAI", title="Optical Flow", flow_vectors=flow_vectors)

### *save_flow*
This function takes as input the path for the NetCDF image and the (already calculated) flow and creates a new NetCDF file with two new variables containing the flow vector components.

In [4]:
def save_flow(file_path, output_path="output.nc", lat_range=None, lon_range=None, 
              flow_vectors=None, flow_vectors_m=None, flow_vectors_f=None, mask_data=False):
    """
    Saves the original NetCDF data and optionally overlays two sets of flow vectors as new data variables into another NetCDF file,
    masking the flow data based on 'fai_anomaly' being zero if mask_data is True.
    """
    # Load the NetCDF data
    data = xr.open_dataset(file_path)
    
    # Subset the data based on provided latitude and longitude ranges
    if lat_range and 'latitude' in data.coords:
        data = data.sel(latitude=slice(*lat_range))
    if lon_range and 'longitude' in data.coords:
        data = data.sel(longitude=slice(*lon_range))

    # Prepare the mask based on 'fai_anomaly' if mask_data is True
    mask = data['fai_anomaly'] != 0 if mask_data else None

    # Function to prepare flow data
    def prepare_flow_data(flow_data, mask, mask_data):
        flow_u, flow_v = flow_data
        if mask_data:
            flow_u = xr.where(mask, flow_u, 0)
            flow_v = xr.where(mask, flow_v, 0)
        return flow_u, flow_v

    # Include the flow vectors as new data variables if provided
    if flow_vectors:
        flow_u, flow_v = prepare_flow_data(flow_vectors, mask, mask_data)
        data['flow_u'] = xr.DataArray(flow_u, dims=("latitude", "longitude"), coords={"latitude": data.latitude, "longitude": data.longitude})
        data['flow_v'] = xr.DataArray(flow_v, dims=("latitude", "longitude"), coords={"latitude": data.latitude, "longitude": data.longitude})
        
    if flow_vectors_m:
        flow_u_m, flow_v_m = prepare_flow_data(flow_vectors_m, mask, mask_data)
        data['flow_u_m'] = xr.DataArray(flow_u_m, dims=("latitude", "longitude"), coords={"latitude": data.latitude, "longitude": data.longitude})
        data['flow_v_m'] = xr.DataArray(flow_v_m, dims=("latitude", "longitude"), coords={"latitude": data.latitude, "longitude": data.longitude})
        
    if flow_vectors_f:
        flow_u_f, flow_v_f = prepare_flow_data(flow_vectors_f, mask, mask_data)
        data['flow_u_f'] = xr.DataArray(flow_u_f, dims=("latitude", "longitude"), coords={"latitude": data.latitude, "longitude": data.longitude})
        data['flow_v_f'] = xr.DataArray(flow_v_f, dims=("latitude", "longitude"), coords={"latitude": data.latitude, "longitude": data.longitude})

    # Save the modified dataset to a new NetCDF file
    data.to_netcdf(output_path)

In [ ]:
# Test on Antilles
if __name__ == "__main__":
    prev_nc = '/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Processed_ABI_Averages/Processed_algae_distribution_20220723.nc'
    next_nc = '/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Processed_ABI_Averages/Processed_algae_distribution_20220724.nc'
    
    flow_vectors = calculate_deepflow(prev_nc, next_nc)
    # Example usage
    save_flow(prev_nc, output_path="/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Antilles_with_flow.nc", flow_vectors=flow_vectors)

In [ ]:
# Test on Atlantic
if __name__ == "__main__":
    prev_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Averages_Binarized/Processed_algae_distribution_20220723.nc"
    next_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Averages_Binarized/Processed_algae_distribution_20220724.nc"
    
    flow_vectors = calculate_deepflow(prev_nc, next_nc)
    # Example usage
    save_flow(prev_nc, 'fai_anomaly', output_path="/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Atlantic_with_flow.nc", flow_vectors=flow_vectors)

In [ ]:
# Test on Filtered Atlantic
if __name__ == "__main__":
    prev_nc = "/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Filtered/Filtered_algae_distribution_20220723.nc"
    next_nc = "/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Filtered/Filtered_algae_distribution_20220724.nc"
    
    flow_vectors = calculate_deepflow(prev_nc, next_nc)
    # Example usage
    save_flow(prev_nc, 'fai_anomaly', output_path="/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Filtered/Atlantic_with_flow.nc", flow_vectors=flow_vectors)

### *visualize_quiver* 
Takes as input a NetCDF file with the added variables **flow_u** and **flow_v** and uses them to overlay the vectors on the image.

In [ ]:
def visualize_quiver(file_path, variable_key="fai_anomaly", quiver_step=10, quiver_scale=100, save_path=None, mask_data=True):
    """
    Visualizes the optical flow vectors on top of the variable image from a NetCDF file.
    Only shows vectors where the data is non-zero if masking is enabled.

    Parameters:
    - file_path (str): Path to the NetCDF file.
    - variable_key (str): Key for the variable of interest (typically the image data).
    - quiver_step (int): Step size for downsampling the quiver plot to reduce vector density.
    - quiver_scale (int): Scaling factor for the vectors to control their size.
    - save_path (str, optional): Path to save the figure as a high-resolution PNG image. If None, the image is not saved.
    - mask_data (bool, optional): If True, vectors are only displayed where the data is non-zero.
    """
    data = xr.open_dataset(file_path)

    # Increase figure size for better resolution in the saved image
    fig, ax = plt.subplots(figsize=(25, 20), subplot_kw={'projection': ccrs.PlateCarree()})
    data[variable_key].plot(ax=ax, x='longitude', y='latitude', transform=ccrs.PlateCarree(), cmap='gray', add_colorbar=False)

    # Calculate the step size for displaying vectors
    skip = (slice(None, None, quiver_step), slice(None, None, quiver_step))
    X, Y = np.meshgrid(data.longitude, data.latitude)
    U = data['flow_u'].values
    V = data['flow_v'].values

    if mask_data:
        # Mask where the data is zero
        mask = data[variable_key].values != 0
        masked_X = X[skip][mask[skip]]
        masked_Y = Y[skip][mask[skip]]
        masked_U = U[skip][mask[skip]]
        masked_V = V[skip][mask[skip]]
    else:
        masked_X = X[skip]
        masked_Y = Y[skip]
        masked_U = U[skip]
        masked_V = V[skip]

    # Overlay the flow vectors
    ax.quiver(masked_X, masked_Y, masked_U, masked_V, color='red', scale=quiver_scale)

    plt.title('Optical Flow on Image')

    # Save the figure if a save_path is provided
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close(fig)  # Close the figure to free up memory
    else:
        plt.show()

In [ ]:
# Antilles
if __name__ == "__main__":
    visualize_quiver("/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Antilles_with_flow.nc", 'fai_anomaly', quiver_step=12, quiver_scale=500, mask_data=False)

In [ ]:
# Antilles (masked)
if __name__ == "__main__":
    visualize_quiver("/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Antilles_with_flow.nc", 'fai_anomaly', quiver_step=5, quiver_scale=1000, mask_data=True)

In [ ]:
# Atlantic
if __name__ == "__main__":
    visualize_quiver("/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Atlantic_with_flow.nc", 'fai_anomaly', quiver_step=45, quiver_scale=3000, save_path=None)

In [ ]:
# Atlantic
if __name__ == "__main__":
    visualize_quiver("/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Atlantic_with_flow.nc", 'fai_anomaly', quiver_step=15, quiver_scale=5000, save_path='/home/yahia/Bureau/atlantic_flow_masked', mask_data=True)
    # visualize_quiver("/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Atlantic_with_flow.nc", 'fai_anomaly', quiver_step=10, quiver_scale=5000, save_path=None, mask_data=True)

### *visualize_flow*
Similar to *visualize_quiver*, but uses custom vectors instead of quiver.

In [ ]:
def visualize_flow(file_path, variable_key="fai_anomaly", quiver_step=10, line_width=0.5, quiver_scale=1.0, save_path=None, mask_data=True):
    data = xr.open_dataset(file_path)

    fig, ax = plt.subplots(figsize=(20, 16), subplot_kw={'projection': ccrs.PlateCarree()})
    data[variable_key].plot(ax=ax, x='longitude', y='latitude', transform=ccrs.PlateCarree(), cmap='gray', add_colorbar=False)

    # Generate meshgrid for coordinates and vector components, applying downsampling by quiver_step
    X, Y = np.meshgrid(data.longitude[::quiver_step], data.latitude[::quiver_step])
    U = data['flow_u'].values[::quiver_step, ::quiver_step]
    V = data['flow_v'].values[::quiver_step, ::quiver_step]

    if mask_data:
        mask = data[variable_key].values[::quiver_step, ::quiver_step] != 0
        X, Y, U, V = X[mask], Y[mask], U[mask], V[mask]

    # Scale the vectors using quiver_scale
    U, V = U * quiver_scale, V * quiver_scale

    # Calculate end points of vectors
    end_X, end_Y = X + U, Y + V
    lines = np.array([[[x, y], [ex, ey]] for x, y, ex, ey in zip(X.flatten(), Y.flatten(), end_X.flatten(), end_Y.flatten())])

    # Create a LineCollection from the arrays of line segments
    lc = LineCollection(lines, colors='red', linewidths=line_width, transform=ccrs.PlateCarree())
    ax.add_collection(lc)

    plt.title('Optical Flow on Image')
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close(fig)  # Close the figure to free up memory
    else:
        plt.show()

In [ ]:
# Antilles
if __name__ == "__main__":
    visualize_flow("/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Antilles_with_flow.nc", 'fai_anomaly', quiver_step=12, quiver_scale=0.005, mask_data=False)

## DeepFlowing Atlantic

In [ ]:
if __name__ == "__main__":
    start_time = time.time()  # Record the start time

    prev_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Filtered/Filtered_algae_distribution_20220723.nc"
    next_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Filtered/Filtered_algae_distribution_20220724.nc"
    
    # Calculate flow vectors for each variable key
    flow_vectors = calculate_deepflow(prev_nc, next_nc, variable_key="fai_anomaly")
    flow_vectors_m = calculate_deepflow(prev_nc, next_nc, variable_key="masked_land")
    flow_vectors_f = calculate_deepflow(prev_nc, next_nc, variable_key="filtered")
    
    # Save the flow data
    save_flow(prev_nc, output_path="/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Filtered_with_flow.nc",
              flow_vectors=flow_vectors, flow_vectors_f=flow_vectors_f, flow_vectors_m=flow_vectors_m)

    end_time = time.time()  # Record the end time after operations are complete
    print(f"Elapsed time: {end_time - start_time:.2f} seconds")  # Print the elapsed time

Sequential: 
- 1st run: 280.08s
- 2nd run (with d_lon, d_lat loop): 446.59s (loop 1: 35.51s , loop 2: 28.19s, loop 3: 28.54s)
- 3rd run: 224.53s (no loop)
Parallel: 

#### DeepFlow (Masked)

In [ ]:
if __name__ == "__main__":
    prev_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Filtered/Filtered_algae_distribution_20220723.nc"
    next_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Filtered/Filtered_algae_distribution_20220724.nc"
    flow_vectors = calculate_deepflow(prev_nc, next_nc, variable_key="fai_anomaly")
    flow_vectors_m = calculate_deepflow(prev_nc, next_nc, variable_key="masked_land")
    flow_vectors_f = calculate_deepflow(prev_nc, next_nc, variable_key="filtered")
    save_flow(prev_nc, output_path="/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Filtered_with_flow_masked.nc", flow_vectors=flow_vectors, flow_vectors_f=flow_vectors_f,
             flow_vectors_m=flow_vectors_m, mask_data=True)

#### Sub-daily

In [ ]:
if __name__ == "__main__":
    prev_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Filtered/Filtered_algae_distribution_20220723.nc"
    next_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Filtered/Filtered_algae_distribution_20220724.nc"
    
    flow_vectors = calculate_deepflow(prev_nc, next_nc, variable_key="fai_anomaly", time_interval=14400)
    # flow_vectors_m = calculate_deepflow(prev_nc, next_nc, variable_key="masked_land")
    # flow_vectors_f = calculate_deepflow(prev_nc, next_nc, variable_key="filtered")
    save_flow(prev_nc, output_path="/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Filtered_with_flow_4h.nc", flow_vectors=flow_vectors)

## Farnebacking Atlantic

In [ ]:
if __name__ == "__main__":
    prev_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Filtered/Filtered_algae_distribution_20220723.nc"
    next_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Filtered/Filtered_algae_distribution_20220724.nc"
    
    flow_vectors = calculate_farneback(prev_nc, next_nc, variable_key="fai_anomaly")
    flow_vectors_m = calculate_farneback(prev_nc, next_nc, variable_key="masked_land")
    flow_vectors_f = calculate_farneback(prev_nc, next_nc, variable_key="filtered")
    save_flow(prev_nc, output_path="/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Filtered_with_farneback.nc", flow_vectors=flow_vectors, flow_vectors_f=flow_vectors_f,
             flow_vectors_m=flow_vectors_m)

#### Farneback (Masked)

In [ ]:
if __name__ == "__main__":
    prev_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Filtered/Filtered_algae_distribution_20220723.nc"
    next_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Filtered/Filtered_algae_distribution_20220724.nc"
    flow_vectors = calculate_farneback(prev_nc, next_nc, variable_key="fai_anomaly")
    flow_vectors_m = calculate_farneback(prev_nc, next_nc, variable_key="masked_land")
    flow_vectors_f = calculate_farneback(prev_nc, next_nc, variable_key="filtered")
    save_flow(prev_nc, output_path="/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Filtered_with_farneback_masked.nc", flow_vectors=flow_vectors, flow_vectors_f=flow_vectors_f,
             flow_vectors_m=flow_vectors_m, mask_data=True)

## LKing Atlantic

In [ ]:
if __name__ == "__main__":
    prev_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Filtered/Filtered_algae_distribution_20220723.nc"
    next_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Filtered/Filtered_algae_distribution_20220724.nc"
    
    flow_vectors = calculate_lk(prev_nc, next_nc, variable_key="fai_anomaly")
    flow_vectors_m = calculate_lk(prev_nc, next_nc, variable_key="masked_land")
    flow_vectors_f = calculate_lk(prev_nc, next_nc, variable_key="filtered")
    save_flow(prev_nc, output_path="/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Filtered_with_lk.nc", flow_vectors=flow_vectors, flow_vectors_f=flow_vectors_f,
             flow_vectors_m=flow_vectors_m)

## Block-Matching Atlantic

In [12]:
if __name__ == "__main__":
    prev_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Filtered/Filtered_algae_distribution_20220723.nc"
    next_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Filtered/Filtered_algae_distribution_20220724.nc"
    # 197 = 86 + 25 + 86
    flow_vectors = calculate_block_matching(prev_nc, next_nc, variable_key="fai_anomaly", block_size=(10, 10), search_area=(50, 50)) 
    flow_vectors_f = calculate_block_matching(prev_nc, next_nc, variable_key="filtered", block_size=(10, 10), search_area=(50, 50))
    save_flow(prev_nc, output_path="/home/yahia/Documents/Jupyter/Sargassum/Images/Test/Block.nc", flow_vectors=flow_vectors, flow_vectors_f=flow_vectors_f,
              mask_data=True)

## Size Matching

### *calculate_flow_and_update_nc*

In [34]:
def calculate_flow_and_update_nc(nc_file_day1, nc_file_day2, output_nc_file, dt=86400, max_pixel_move=100):
    ds1 = xr.open_dataset(nc_file_day1)
    ds2 = xr.open_dataset(nc_file_day2)
    
    binarized1 = ds1["filtered"].values
    binarized2 = ds2["filtered"].values
    
    labels1, _ = label(binarized1)
    labels2, _ = label(binarized2)
    
    latitudes = ds1.latitude.values
    longitudes = ds1.longitude.values
    
    flow_u = np.zeros_like(binarized1, dtype=np.float32)
    flow_v = np.zeros_like(binarized1, dtype=np.float32)
    
    props1 = regionprops(labels1)
    props2 = regionprops(labels2)
    for prop1 in props1:
        centroid1 = prop1.centroid
        best_match, min_dist = None, np.inf
        for prop2 in props2:
            centroid2 = prop2.centroid
            dist = np.linalg.norm(np.array(centroid1) - np.array(centroid2))
            if dist <= max_pixel_move and dist < min_dist:
                min_dist, best_match = dist, prop2

        if best_match:
            dx = best_match.centroid[1] - prop1.centroid[1]
            dy = best_match.centroid[0] - prop1.centroid[0]
            lon1, lat1 = longitudes[int(prop1.centroid[1])], latitudes[int(prop1.centroid[0])]
            lon2, lat2 = longitudes[int(prop1.centroid[1] + dx)], latitudes[int(prop1.centroid[0] + dy)]
            
            distance_x = haversine(lon1, lat1, lon2, lat1) * np.sign(dx) * 1000
            distance_y = haversine(lon1, lat1, lon1, lat2) * np.sign(dy) * 1000
            flow_u[labels1 == prop1.label] = distance_x / dt
            flow_v[labels1 == prop1.label] = distance_y / dt

    new_ds = xr.Dataset({
        'flow_u': (('latitude', 'longitude'), flow_u),
        'flow_v': (('latitude', 'longitude'), flow_v)
    }, coords={'latitude': ds1.latitude, 'longitude': ds1.longitude})
    new_ds.to_netcdf(output_nc_file)


In [35]:
if __name__ == "__main__":
    prev_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Filtered/Filtered_algae_distribution_20220723.nc"
    next_nc = "/media/yahia/ballena/ABI/NetCDF/Atlantic/Filtered/Filtered_algae_distribution_20220724.nc"
    output_path="/home/yahia/Documents/Jupyter/Sargassum/Images/Test/matching.nc"
    calculate_flow_and_update_nc(prev_nc, next_nc, output_path)